In [2]:
import numpy as np
import itertools
import math
#from wolframclient.evaluation import WolframLanguageSession
#from wolframclient.language import wl, wlexpr
import heapq
import pandas as pd

# Problem statement

<p float="left">
  <img src="https://i.imgur.com/nDe6qQB.png" width="500" />
  <img src="https://i.imgur.com/o7ruAe7.png" width="600" />
</p>

<img src="https://i.imgur.com/3R1W4Ln.png" width="800" />

Here's a breakdown of how your profit from an expedition will be computed:
Every spot has its **treasure multiplier** (up to 100) and the number of **hunters** (up to 8). The spot's total treasure is the product of the **base treasure** (7500, same for all spots) and the spot's specific treasure multiplier. However, the resulting amount is then divided by the sum of the hunters and the percentage of all the expeditions (from other players) that took place there. For example, if a field has 5 hunters, and 10% of all the expeditions (from all the other players) are also going there, the prize you get from that field will be divided by 15. After the division, **expedition costs** apply (if there are any), and profit is what remains.

Second and third expeditions are optional: you are not required to do all 3. Fee for embarking upon a second expedition is 25 000, and for third it's 75 000. Order of submitted expeditions does not matter for grading.

# Solution

The profit for an expedition to a destination with $M$ as multiplier, $H$ hunters, and share $p\in [0,1]$ of all expeditions is:
$$\begin{cases} 
7500\frac M{H+100 p} &\text{if this is the first expedition,}  
\\ 7500\frac M{H+100 p} - 25000 &\text{if this is the second expedition,} 
\\ 7500\frac M{H+100 p} - 75000 &\text{if this is the third expedition.} 
\end{cases} 
$$
The second expedition is profitable if and only if $p < \frac{3M - 10 H}{1000}$, and the third is when $p < \frac{M - 10 H}{1000}$.  
Let us see from the data how stringent these conditions are on $p$.

In [6]:
arr = np.array([(10,1), (80,6), (37,3), (90,10), (31,2),(17,1),(50,4),(20,2),(73,4),(89,8)])

The next cell computes the array with entries $\frac{3M - 10 H}{1000}$.

In [8]:
thresh = np.zeros(10)
for i in range(10):
    mult, hunt = arr[i]
    thresh[i] = (mult/5 - hunt)
with np.printoptions(precision=3, suppress=True):
    print(thresh)

[ 1.  10.   4.4  8.   4.2  2.4  6.   2.  10.6  9.8]


The entries around the center of the array fluctuate around $20\%$, hence profitability of a second expedition to one of these destinations is likely.

Next, we do the same for $\frac{M - 10 H}{1000}$.

In [4]:
thresh = np.zeros((5, 5))
for i in range(5):
    for j in range(5):
        mult, hunt = arr[i, j]
        thresh[i, j] = (mult - 10*hunt)/1000
with np.printoptions(precision=3, suppress=True):
    print(thresh)

[[ 0.004  0.03   0.011  0.001  0.02 ]
 [ 0.017  0.032  0.037  0.03   0.005]
 [ 0.033  0.039  0.02   0.02  -0.003]
 [ 0.027  0.033  0.035  0.029  0.015]
 [-0.008 -0.003  0.012 -0.005  0.   ]]


Negative entries correspond to third expeditions that will systematically result in a loss.  
The other entries are all smaller than $4\%$, with many being less than $2\%$.

## 1. Maximin approach

The shares are computed once the choices of destinations are collected over all the teams.  
However, since the number of teams is large, our choice has very little impact on the shares.

One way to tackle the uncertainty in the share $p$ is to consider the most pessimistic setting where $p$ is chosen by a malicious adversary in order to make our profit as small as possible.  
For a choice of a single destination $(M,H)$, our profit $\pi(M,H,p)$ rewrites $\pi(M,H,p^*(M,H))$ where $p^*(M,H)$ is a solution of 
$$\min_{p\in [0,1]}  \frac {7500M}{H+100 p}.$$
For a single expedition, the complete optimization problem is therefore 
$$\max_{(M,H)} \min_{p\in [0,1]}  \frac {7500M}{H+100 p} = \max_{(M,H)}  \frac {7500M}{H+100}$$

In [5]:
def maximin1():
    """Solves the maximin optimization problem for a single expedition.

    Parameters
    ----------
    
    Returns
    -------
    argmax : list of tuple
        Maximizers.
    max_val : float
        Maximal profit.
        
    """
    max_val = float('-inf')
    argmax = []
    for mult, hunt in arr.reshape((25, 2)):
        val = mult / (hunt + 100) * 7500
        if math.isclose(val, max_val):
            argmax.append((mult, hunt))
        elif val > max_val:
            argmax = [(mult, hunt)]
            max_val = val
    return (argmax, max_val)

In [6]:
maximin1()

([(100, 8)], 6944.444444444444)

For a total of two expeditions, the optimization problem is 
$$\max_{(M_1,H_1), (M_2,H_2)} \quad \min_{\substack{p_1, p_2\in [0,1]\\ p_1+p_2\leq 1}}  \frac {7500M_1}{H_1+100 p_1}+\frac {7500M_2}{H_2+100 p_2}-25000.$$
Since the inner minimization problem is no longer trivial, we make a call to the Mathematica function `NMinimize` to solve it.

In [7]:
session = WolframLanguageSession()

def maximize2():
    """Solves the maximin optimization problem for two expeditions. Returns only one solution, there may be other.

    Parameters
    ----------
    
    Returns
    -------
    argmax1 : tuple
        First expedition.
    argmax2 : tuple
        Second expedition.
    max_val : float
        Maximal profit.
        
    """
    max_val = float('-inf')
    max_mult1, max_hunt1 = None, None
    max_mult2, max_hunt2 = None, None
    for (mult1, hunt1), (mult2, hunt2) in itertools.combinations(arr.reshape((25, 2)), 2):
        val_temp = session.evaluate(wlexpr(f'NMinimize[{{{mult1}/({hunt1} + 100*p1) + {mult2}/({hunt2} + 100*p2), 0 <= p1, 0 <= p2, p1 + p2 <= 1}}, {{p1, p2}}]'))[0]
        val = -25000 + val_temp * 7500
        if math.isclose(val, max_val):
            print("collision")
        if val > max_val:
            max_mult1, max_hunt1 = mult1, hunt1
            max_mult2, max_hunt2 = mult2, hunt2
            max_val = val
    return ((max_mult1, max_hunt1), (max_mult2, max_hunt2), max_val)

In [8]:
maximize2()

((89, 5), (100, 8), 67.23154797644384)

When proceeding with two expeditions, the maximin profit is only $\approx 67$.  
From the pessimistic maximin standpoint, it is better to go for a single expedition.

For a total of three expeditions, the optimization problem is 
$$\max_{(M_1,H_1), (M_2,H_2), (M_3,H_3)} \quad \min_{\substack{p_1, p_2, p_3\in [0,1]\\ p_1+p_2+p_3\leq 1}}  \frac {7500M_1}{H_1+100 p_1}+\frac {7500M_2}{H_2+100 p_2}
+\frac {7500M_3}{H_3+100 p_3}-100000.$$
We proceed identically as above.

In [9]:
def maximize3():
    """Solves the maximin optimization problem for three expeditions. Returns only one solution, there may be other.

    Parameters
    ----------
    
    Returns
    -------
    argmax1 : tuple
        First expedition.
    argmax2 : tuple
        Second expedition.
    argmax3 : tuple
        Third expedition.
    max_val : float
        Maximal profit.
        
    """
    max_val = float('-inf')
    max_mult1, max_hunt1 = None, None
    max_mult2, max_hunt2 = None, None
    max_mult3, max_hunt3 = None, None
    for (mult1, hunt1), (mult2, hunt2), (mult3, hunt3) in itertools.combinations(arr.reshape((25, 2)), 3):
        val_temp = session.evaluate(wlexpr(f'NMinimize[{{{mult1}/({hunt1} + 100*p1) + {mult2}/({hunt2} + 100*p2) + {mult3}/({hunt3} + 100*p3), 0<=p1, 0<=p2, 0<=p3, p1+p2+p3 <= 1}}, {{p1,p2,p3}}]'))[0]
        val = -100000 + val_temp * 7500
        if math.isclose(val, max_val):
            print("collision")
        if val > max_val:
            max_mult1, max_hunt1 = mult1, hunt1
            max_mult2, max_hunt2 = mult2, hunt2
            max_mult3, max_hunt3 = mult3, hunt3
            max_val = val
    return ((max_mult1, max_hunt1), (max_mult2, max_hunt2), (max_mult3, max_hunt3), max_val)

In [10]:
maximize3()

((87, 5), (89, 5), (100, 8), -47422.72209123209)

## 2. Optimization given a prior

A less pessimistic approach is to model the shares *a priori*.
For each destination, the corresponding share is no longer unknown. 
Instead we posit a value for this share.  
Shares are nonnegative and sum to $1$, thus shares are modelled by a probability distribution that we call a prior.

Given a prior, the optimization problem is 
$$\max_{L\in \{1,2,3\}} \quad \max_{(M_i,H_i,p_i)} \text{fee}(L) + \sum_{i=1}^L \frac{7500 M_i}{H_i + 100p_i}   $$

### a. Natural prior

The ratio $M/H$ seems like a plausible proxy for the attractiveness of a destination.  
We compute the array `ratios` with entries $\frac MH$ and normalize it to obtain the prior array `shares` on the distribution of expeditions.

In [9]:
ratios = np.zeros(10)
for i in range(10):
    mult, hunt = arr[i]
    ratios[i] = mult/hunt
with np.printoptions(precision=2, suppress=True):
    print("Ratios:", ratios, sep='\n')

Ratios:
[10.   13.33 12.33  9.   15.5  17.   12.5  10.   18.25 11.12]


In [10]:
shares = ratios / np.sum(ratios)
with np.printoptions(precision=3, suppress=True):
    print("Natural prior:", shares, sep='\n')

Natural prior:
[0.077 0.103 0.096 0.07  0.12  0.132 0.097 0.077 0.141 0.086]


`shares[0, 0] = 0.035` means that we posit that $3.5\%$ of all expeditions will go to the destination with multiplier $24$ and $2$ hunters.

The difference between larger and smaller values in `ratios` can be accentuated by elevating all the entries of `ratios` to some power.

In [13]:
ratios = np.zeros((5, 5))
for i in range(5):
    for j in range(5):
        mult, hunt = arr[i, j]
        ratios[i, j] = mult/hunt
ratios = ratios**5
shares = ratios / np.sum(ratios)
with np.printoptions(precision=3, suppress=True):
    print("Natural prior with exponent 5:", shares, sep='\n')

Natural prior with exponent 5:
[[0.014 0.089 0.026 0.007 0.041]
 [0.051 0.065 0.087 0.057 0.012]
 [0.11  0.097 0.017 0.019 0.002]
 [0.047 0.069 0.077 0.054 0.027]
 [0.    0.003 0.02  0.001 0.005]]


The next cell solves the optimization problem by iterating over the whole state space.

In [11]:
def fee(n):
    """Compute the fee for a total of n expeditions.

    Parameters
    ----------
    n : int
        Number of expeditions.
    
    Returns
    -------
    float
        Fee.
    """
    if n == 1:
        return 0
    if n == 2:
        return -50000

def payoff(mults, hunts, shares):
    """Compute the final profit after the expeditions.

    Parameters
    ----------
    mults : list of int
        Multipliers for each destination.
    hunts : list of int
        Hunters for each destination.
    shares : list of int
        Shares for each destination.
    
    Returns
    -------
    float
        Profit.
    """
    return 10000 * sum([mult/(hunt + 100*share) for (mult, hunt, share) in zip(mults, hunts, shares)]) + fee(len(mults))

def maximize_prior_top(shares, k):
    """Given the prior, compute solutions that yield top k profits.

    Parameters
    ----------
    shares : list of int
        Shares for each destination.
    k : int
        Number of solutions
    
    Returns
    -------
    list of tuple
        Top k profits and optimal expeditions.
    """
    datas = [(mult, hunt, share) for ((mult, hunt), share) in zip(arr.reshape((10, 2)), shares.reshape(10))]
    heap = []
    iterables = [itertools.combinations(datas, n_exp) for n_exp in range(1, 3)]
    for (i, data) in enumerate(itertools.chain.from_iterable(iterables)):
        mults = [tupl[0] for tupl in data]
        hunts = [tupl[1] for tupl in data]
        shares = [tupl[-1] for tupl in data]
        val = payoff(mults, hunts, shares)
        expeditions = list(zip(mults, hunts))
        if i < k:
            heapq.heappush(heap, (val, expeditions))
        elif val > heap[0][0]:
            heapq.heappop(heap)
            heapq.heappush(heap, (val, expeditions))
    return sorted(heap, reverse=True)

Next, we consider the priors obtained by elevating the entries of `ratios` to an exponent `i`.   
When `i=0`, the prior is uniformly distributed over all destinations.

In [13]:
shares = ratios**i
shares = shares / np.sum(shares)
print(shares)
res = maximize_prior_top(shares, 5)
print("Exponent:", i, "Profit:", f"{res[0][0]:.2f}", "Optimal expeditions:", res[0][1],res[1][1],res[2][1],res[3][1],res[4][1])



Exponent: 0 Profit: 52142.86 Optimal expeditions: [(80, 6), (73, 4)] [(73, 4)] [(73, 4), (89, 8)] [(80, 6)] [(80, 6), (89, 8)]
Exponent: 1 Profit: 56566.66 Optimal expeditions: [(90, 10), (89, 8)] [(89, 8)] [(90, 10)] [(80, 6), (89, 8)] [(80, 6), (90, 10)]
Exponent: 2 Profit: 70658.70 Optimal expeditions: [(90, 10), (89, 8)] [(90, 10)] [(80, 6), (90, 10)] [(89, 8)] [(80, 6), (89, 8)]
Exponent: 3 Profit: 85605.23 Optimal expeditions: [(90, 10), (89, 8)] [(80, 6), (90, 10)] [(90, 10)] [(80, 6), (89, 8)] [(89, 8)]
Exponent: 4 Profit: 100070.58 Optimal expeditions: [(90, 10), (89, 8)] [(80, 6), (90, 10)] [(80, 6), (89, 8)] [(90, 10)] [(90, 10), (50, 4)]
Exponent: 5 Profit: 112950.96 Optimal expeditions: [(90, 10), (89, 8)] [(80, 6), (90, 10)] [(80, 6), (89, 8)] [(90, 10), (50, 4)] [(90, 10), (20, 2)]
Exponent: 6 Profit: 123630.03 Optimal expeditions: [(90, 10), (89, 8)] [(80, 6), (89, 8)] [(20, 2), (89, 8)] [(80, 6), (90, 10)] [(50, 4), (89, 8)]
Exponent: 7 Profit: 131966.18 Optimal expedi

In [23]:
pnl_list = None
initial_scores = np.ones(10)
for i in range(3):
    if i == 0:
        pass
    else:
        initial_scores = pnl_list
    shares = initial_scores / np.sum(initial_scores)
    print(shares)
    pnl_list = [10000 * (mult/(hunt + 100*share)) for (mult, hunt, share) in zip(arr[:, 0], arr[:, 1], shares)]
    print(pnl_list)


[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[9090.90909090909, 50000.0, 28461.53846153846, 45000.0, 25833.333333333336, 15454.545454545454, 35714.28571428572, 16666.666666666668, 52142.857142857145, 49444.444444444445]
[0.02773237 0.15252804 0.08682365 0.13727524 0.07880615 0.04714503
 0.1089486  0.05084268 0.15906496 0.15083328]
[26502.442851498818, 37642.09197368602, 31671.66837931395, 37930.63354491098, 31374.564102001714, 29748.868637851676, 33568.62715232435, 28231.568980264517, 36671.44722913921, 38555.965044611796]
[0.0798512  0.11341468 0.09542594 0.11428405 0.09453078 0.0896326
 0.10114143 0.08506101 0.11049015 0.11616816]
[11129.512268343888, 46132.19567129162, 29499.47894209314, 42000.326277772234, 27066.960479009253, 17062.688880551093, 35425.45853001899, 19036.557753786976, 48508.15728575148, 45369.238825199944]


In [ ]:
another_ratios = np.zeros(10)
for i in range(10):
    mult, hunt = arr[i]
    another_ratios[i] = (mult - 10*hunt)/1000
with np.printoptions(precision=2, suppress=True):
    print("Ratios:", ratios, sep='\n')

In [14]:
for i in range(10):
    shares = ratios**i
    shares = shares / np.sum(shares)
    print(shares)
    res = maximize_prior_top(shares, 5)
    print("Exponent:", i, "Profit:", f"{res[0][0]:.2f}", "Optimal expeditions:", res[0][1],res[1][1],res[2][1],res[3][1],res[4][1])

[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
Exponent: 0 Profit: 52142.86 Optimal expeditions: [(80, 6), (73, 4)] [(73, 4)] [(73, 4), (89, 8)] [(80, 6)] [(80, 6), (89, 8)]
[0.07749435 0.1033258  0.09557636 0.06974491 0.12011624 0.13174039
 0.09686794 0.07749435 0.14142719 0.08621246]
Exponent: 1 Profit: 56566.66 Optimal expeditions: [(90, 10), (89, 8)] [(89, 8)] [(90, 10)] [(80, 6), (89, 8)] [(80, 6), (90, 10)]
[0.057038   0.1014009  0.08676114 0.04620078 0.13703381 0.16483983
 0.08912188 0.057038   0.1899722  0.07059344]
Exponent: 2 Profit: 70658.70 Optimal expeditions: [(90, 10), (89, 8)] [(90, 10)] [(80, 6), (90, 10)] [(89, 8)] [(80, 6), (89, 8)]
[0.03996692 0.09473641 0.07497943 0.02913589 0.14883182 0.19635749
 0.0780604  0.03996692 0.24293457 0.05503016]
Exponent: 3 Profit: 85605.23 Optimal expeditions: [(90, 10), (89, 8)] [(80, 6), (90, 10)] [(90, 10)] [(80, 6), (89, 8)] [(89, 8)]
[0.02679475 0.08468465 0.06199713 0.01758004 0.15465948 0.22379245
 0.06541687 0.02679475 0.29723587 0

### b. Online poll as prior

Another prior can be obtained by exploiting the data from this [online poll](https://docs.google.com/spreadsheets/d/16xO1L8NukksiQCiutmdsTHh51_HXxqulelJez653hjY/edit#gid=0).  
Note that the sample size (58) is quite small, and this data is clearly biased and not quite reliable.

In [16]:
df = pd.read_csv('data/poll.csv', sep=';', header=None)
df = df.to_numpy().flatten()
df = df[~np.isnan(df)]
counts = {}
for n in df:
    counts[n] = counts.get(n, 0) + 1
shares = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        mult = arr[i, j][0]
        shares[i, j] = counts.get(mult, 0)
shares = shares / np.sum(shares)
with np.printoptions(precision=3, suppress=True):
    print("Prior from online poll:", shares, sep='\n')

Prior from online poll:
[[0.015 0.045 0.038 0.015 0.038]
 [0.03  0.045 0.053 0.06  0.03 ]
 [0.068 0.083 0.045 0.053 0.03 ]
 [0.068 0.053 0.053 0.06  0.053]
 [0.    0.008 0.03  0.    0.03 ]]


In [17]:
maximize_prior_top(shares, 10)

[(103237.0021283065, [(82, 5), (87, 5)]),
 (101775.46366676802, [(82, 5), (85, 5)]),
 (101342.94124219612, [(70, 4), (82, 5)]),
 (100692.30769230767, [(87, 5), (85, 5)]),
 (100313.92520522956, [(82, 5), (83, 5)]),
 (100259.7852677358, [(70, 4), (87, 5)]),
 (99605.99251292186, [(82, 5), (100, 8)]),
 (99230.76923076922, [(87, 5), (83, 5)]),
 (98798.24680619733, [(70, 4), (85, 5)]),
 (98522.83653846155, [(87, 5), (100, 8)])]

### Results 

In order to balance the results for the different priors that I explored, my final choice was two expeditions on the tiles with multipliers $100$ and $82$. 

<img src="https://i.imgur.com/n7q6t4x.png" width="1200" />


Some of the true (i.e., ex post) shares are collected [here](https://docs.google.com/spreadsheets/d/1PlQlcJmFzcFJ_DV62cvzkVgphLiW0A5hLf7IwLlvYJQ/edit#gid=0).

In [18]:
temp = pd.read_csv('data/shares.csv', sep=';', header=None).to_numpy()
dic = {mult:share for (mult, share) in temp}
shares_true = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        shares_true[i, j] = dic[arr[i, j][0]]
shares_true = shares_true/100
with np.printoptions(precision=3, suppress=True):
    print("True shares:", shares_true, sep='\n')

True shares:
[[0.015 0.082 0.019   nan 0.037]
 [0.03  0.062 0.098 0.041 0.012]
 [0.113 0.108 0.049 0.034 0.006]
 [0.046 0.054 0.065 0.054 0.026]
 [  nan   nan 0.019   nan   nan]]


Filling in the NaNs with equal values, the top 10 choices are displayed below.

In [19]:
val_nan = (1-np.nansum(shares_true))/np.sum(np.isnan(shares_true))
maximize_prior_top(np.nan_to_num(shares_true, nan=val_nan),10)

[(106290.59586034171, [(80, 5), (52, 4)]),
 (105375.42951194028, [(90, 7), (52, 4)]),
 (105301.14425749943, [(80, 5), (90, 7)]),
 (103938.17304238147, [(52, 4), (30, 3)]),
 (103892.67636555269, [(35, 3), (52, 4)]),
 (103863.8877879406, [(80, 5), (30, 3)]),
 (103818.39111111182, [(80, 5), (35, 3)]),
 (103583.32345590748, [(41, 3), (52, 4)]),
 (103509.03820146661, [(41, 3), (80, 5)]),
 (102948.72143953915, [(90, 7), (30, 3)])]

The optimal decision was to go for two expeditions at the tiles with multipliers $80$ and $52$.